# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [22]:
%iam_role arn:aws:iam::212430227630:role/LabRole
%region us-east-1
%number_of_workers 2

%idle_timeout 30
%glue_version 4.0
%worker_type G.1X

Current iam_role is arn:aws:iam::212430227630:role/LabRole
iam_role has been set to arn:aws:iam::212430227630:role/LabRole.
Previous region: us-east-1
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: 2
Setting new number of workers to: 2
Current idle_timeout is 30 minutes.
idle_timeout has been set to 30 minutes.
Setting Glue version to: 4.0
Previous worker type: G.1X
Setting new worker type to: G.1X


In [25]:
%extra_py_files s3://cryptoengineer/gluejobs-py-modules/load.py, s3://cryptoengineer/gluejobs-py-modules/storage.py
%additional_python_modules yfinance

Extra py files to be included:
s3://cryptoengineer/gluejobs-py-modules/load.py
s3://cryptoengineer/gluejobs-py-modules/storage.py
Additional python modules to be included:
yfinance


####  Run this cell to set up and start your interactive session.


In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
import boto3

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 30
Session ID: 19eefaae-215f-4152-9a5e-98d264cdaab8
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--extra-py-files s3://cryptoengineer/gluejobs-py-modules/load.py,s3://cryptoengineer/gluejobs-py-modules/storage.py
--additional-python-modules yfinance
Waiting for session 19eefaae-215f-4152-9a5e-98d264cdaab8 to get into ready status...
Session 19eefaae-215f-4152-9a5e-98d264cdaab8 has been created.



## Batch Load - COMMODITIES


In [2]:
from datetime import datetime, timedelta, timezone
import pandas as pd

import load

### Set AWS Storage parameters


In [3]:
BUCKET_NAME = "cryptoengineer"
PREFIX = "datalake/bronze/commodities"

### Load job parameters

In [41]:
glue_client = boto3.client("glue")

if '--WORKFLOW_NAME' in sys.argv and '--WORKFLOW_RUN_ID' in sys.argv:
    print("Running in Glue Workflow")
    
    glue_args = getResolvedOptions(
        sys.argv, ['WORKFLOW_NAME', 'WORKFLOW_RUN_ID']
    )
    
    print("Reading the workflow parameters")
    workflow_args = glue_client.get_workflow_run_properties(
        Name=glue_args['WORKFLOW_NAME'], RunId=glue_args['WORKFLOW_RUN_ID']
    )["RunProperties"]

    
    base= workflow_args['base']
    time_frame = int(workflow_args['time_frame'])
    freq= workflow_args['freq']
    symbols = workflow_args['symbols']
    api_key = workflow_args['api_key']

else:
    try:
        args = getResolvedOptions(sys.argv,
                                  ['JOB_NAME',
                                   'base',
                                   'time_frame',
                                   'freq',
                                   'symbols',
                                   'api_key'])
        base= args['base']
        time_frame = int(args['time_frame'])
        freq = args['freq']
        ##symbols = "EUR,CHF,JPY,CAD,GBP"
        symbols = args['symbols']
        api_key = args['api_key']
        print("Running as Job")        
    except:
        print("Running as Notebook")
        base = "USD"
        time_frame = 43800 #48
        freq= '1day' #'15min'
        symbols = "GCUSD"
        api_key = ""

Running as Notebook


In [ ]:
print("base: ", base)
print("Time Frame: ", time_frame)
print("Frequency: ", freq)
print("Symbols: ", symbols)
print("API Key: ", api_key)

#### Set the start and end dates for the data you want to load

In [43]:
# Start date
start_date = (datetime.utcnow() - timedelta(hours=time_frame)).strftime("%Y-%m-%d")
end_date = datetime.utcnow().strftime("%Y-%m-%d")

print("Start date; ",start_date," End date: ",end_date)

Start date;  2019-09-02  End date:  2024-08-31


## Load the historical commodities- 15min frequency

Set some config values

In [44]:
type='COMMODITIES'
source='FMP'

In [45]:
if freq == '15min':
    print("Reading historical data by frequency")
    df= pd.DataFrame()
    for symbol in symbols.split(","):
        print("Loading: ", symbol)
        symbol_df = load.load_batch_freq_rates(base=base,
                                              start_date=start_date,
                                              end_date=end_date,
                                              freq=freq,
                                              symbol=symbol,
                                              api_key=api_key,
                                              source=source
        )
        # Complete the table schema 
        if len(symbol_df)>0:
            symbol_df = load.set_schema_table(symbol_df, symbol, source, freq, base, type)
            print("Records: ", len(symbol_df))
            df = pd.concat([df, symbol_df])
        else:
            print("No data for: ", symbol)

else:
    print("Reading daily historical data")
    df= pd.DataFrame()
    for symbol in symbols.split(","):
        print("Loading: ", symbol)
        symbol_df = load.load_historical_rates(base=base,
                                              start_date=start_date,
                                              end_date=end_date,
                                              symbol=symbol,
                                              api_key=api_key,
                                              source=source
        )
        # Complete the table schema 
        if len(symbol_df)>0:        
            symbol_df = load.set_schema_table(symbol_df, symbol, source, freq, base, type)
            print("Records: ", len(symbol_df))
            df = pd.concat([df, symbol_df])
        else:
            print("No data for: ", symbol)


Reading daily historical data
Loading:  GCUSD
https://financialmodelingprep.com/api/v3/historical-price-full
https://financialmodelingprep.com/api/v3/historical-price-full/GCUSD?apikey=xGFdE9Ydrcr1oCDJiCjHiZnkqUnQnjaH
Lectura API correcta
Records:  1302


In [46]:
print("Records: ", len(df))

Records:  1302


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1302 entries, 0 to 1301
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   datetime          1302 non-null   object             
 1   open              1302 non-null   float64            
 2   high              1302 non-null   float64            
 3   low               1302 non-null   float64            
 4   close             1302 non-null   float64            
 5   adjClose          1302 non-null   float64            
 6   volume            1302 non-null   int64              
 7   unadjustedVolume  1302 non-null   int64              
 8   change            1302 non-null   float64            
 9   changePercent     1302 non-null   float64            
 10  vwap              1302 non-null   float64            
 11  label             1302 non-null   object             
 12  changeOverTime    1302 non-null   float64            
 13  yea

In [48]:
df.head(1)

     datetime    open  ...   load_date         type
0  2024-08-30  2553.6  ...  2024-08-31  COMMODITIES

[1 rows x 25 columns]


## Append the batch data to RAW table

Set the destination raw table

In [51]:
path=f"s3://{BUCKET_NAME}/{PREFIX}"
print("Path:",path)

Path: s3://cryptoengineer/datalake/raw/commodities


In [52]:
if len(df)>0:
    print("Saving data to: ", path)    
    (
        spark.createDataFrame(df)
        .repartition("load_date")
        .write
        .format("parquet")
        .mode("append")
        .partitionBy(['load_date'])
        .save(path)
    )

/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
